In [46]:
import cv2
# from tensorflow import keras as k

In [47]:
path = r'C:\\Users\\Administrator\\Desktop\elephant\\OCR_test\\20190309162316.png'

def cv_show(img, name):
    cv2.imshow(name, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [48]:
img = cv2.imread(path)
cv_show(img, 'img')

In [66]:
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# print(imgGray.shape)
imgGray_r = cv2.resize(imgGray,(342,514))    #干，shape是按高宽显示，resize是按宽高显示，正好相反，太久没学习忘掉了........
# print(imgGray_r.shape)
cv_show(imgGray_r,'imgGray_r')

In [70]:
imgcropped = imgGray_r[152:495, 67:253]    #先高后宽：从上到下，从左到右
# cv2.putText(imgcropped,"cute lady",(50,150),cv2.FONT_ITALIC,0.7,(50,221,214),3)
cv_show(imgcropped,'imgcropped')